In [1]:
import os
from azureml.core import Workspace, Datastore, Dataset
from azureml.pipeline.wrapper import Module, dsl, Pipeline
from azureml.pipeline.wrapper._dataset import get_global_dataset_by_path
from azureml.core.compute import AmlCompute, ComputeTarget

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception cannot import name '_update_args_and_inputs' from 'azureml.core.script_run_config' (C:\Users\zhohuang\AppData\Roaming\Python\Python37\site-packages\azureml\core\script_run_config.py).


In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

aml_compute_target = "aml-compute"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("Found existing compute target: {}".format(aml_compute_target))
except:
    print("Creating new compute target: {}".format(aml_compute_target))

    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                min_nodes=1,
                                                                max_nodes=4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
DesignerTest-EUS
DesignerTestRG
eastus
4faaaf21-663f-4391-96fd-47197c630979
Found existing compute target: aml-compute


In [3]:
blob_input_data = get_global_dataset_by_path(
    ws, 'Automobile_price_data',
    'GenericCSV/Automobile_price_data_(Raw)')
notebook_module_func = Module.from_notebook(ws, 'authoring/sample_module_from_notebook/from_notebook.ipynb')
module1 = notebook_module_func(input=blob_input_data, param1='aaa0', param2='bbb0',i=1,j=2,flag=False)
module2 = notebook_module_func(input=module1.outputs['output'], param1='aaa1', param2='bbb1',i=2,j=3,flag=True)
steps = [module1, module2]

INFO     - Extracting dsl module info from the notebook
INFO     - Module entry file '_from_notebook.py' is dumped.


In [4]:
pipeline_name = "notebook_test"
pipeline = Pipeline(name=pipeline_name, nodes=steps,
                    outputs={**steps[-1].outputs})
run = pipeline.submit(pipeline_name, "aml-compute")
print(run.id)
run.wait_for_completion()



Submitted PipelineRun 9262aeb0-23a8-4975-b978-eedf02fd0def
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/notebook_test/runs/9262aeb0-23a8-4975-b978-eedf02fd0def?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS
9262aeb0-23a8-4975-b978-eedf02fd0def
PipelineRunId: 9262aeb0-23a8-4975-b978-eedf02fd0def
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/notebook_test/runs/9262aeb0-23a8-4975-b978-eedf02fd0def?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS


<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_dae93185-aa7d-4aa6-ac5e-976479136dd8_widget', env_json='{}', graph_jso…

<RunStatus.failed: 'Failed'>